In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark


In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("profeco data") \
    .getOrCreate()

In [3]:
# read the csv file
df = spark.read.csv('data/all_data.csv',header = True,sep=",")

In [4]:
# peek the data
df.show(1)

+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|            producto|        presentacion|   marca|       categoria|        catalogo|precio|       fechaRegistro|   cadenaComercial|      giro|     nombreComercial|           direccion|          estado|           municipio| latitud|  longitud|
+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|CUADERNO FORMA IT...|96 HOJAS PASTA DU...|ESTRELLA|MATERIAL ESCOLAR|UTILES ESCOLARES|  25.9|2011-05-18 00:00:...|ABASTECEDORA LUMEN|PAPELERIAS|ABASTECEDORA LUME...|CANNES No. 6 ESQ....|DISTRITO FEDERAL|TLALPAN          ...|19.29699|-99.125417|
+-------------------

In [5]:
df.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: string (nullable = true)
 |-- fechaRegistro: string (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



### 1. Procesamiento de los datos 
#### a. ¿Cuántos registros hay?

In [6]:
df.count()

62530715

#### b. ¿Cuántas categorías?

In [7]:
df.select("categoria").distinct().show()

+--------------------+
|           categoria|
+--------------------+
|    MATERIAL ESCOLAR|
|ARTS. PARA EL CUI...|
|PESCADOS Y MARISC...|
|UTENSILIOS DOMEST...|
|           categoria|
|DETERGENTES Y PRO...|
|CARNE Y VISCERAS ...|
|PRODUCTOS DE TEMP...|
|GALLETAS PASTAS Y...|
|  HORTALIZAS FRESCAS|
|                null|
|  DERIVADOS DE LECHE|
|TORTILLAS Y DERIV...|
|GRASAS ANIMALES C...|
|APARATOS ELECTRON...|
|     LEGUMBRES SECAS|
|                CAFE|
|   MUEBLES DE COCINA|
|CARNES FRIAS SECA...|
|CHOCOLATES Y GOLO...|
+--------------------+
only showing top 20 rows



In [8]:
df.select("categoria").distinct().count()

42

#### c. ¿Cuántas cadenas comerciales están siendo monitoreadas?

In [14]:
df.select("cadenaComercial").distinct().count()

706

#### d. ¿Cómo podrías determinar la calidad de los datos? ¿Detectaste algún tipo de inconsistencia o error en la fuente?

Se puede ver que hay algunas inconsistencia con la información, como:
- Valores Nulos
- Nombres que podría unirse en uno; Distrito Federal con México
- Datos que están incompletos o que fueron llenados de manera incorrecta, por ejemplo; en la columna de categoría, aparece la palabra _categoría_ en lugar de un tipo de categoría en específico
- Los tipos de datos no corresponden a la información mostrada, tal es el caso de un float como string, que no permitiría realziar operaciones matemáticas con él.
- Es posible que la información que se tiene esté anidada en varias tablas, eso explicaría por qué algunos títulos de columnas se repiten como valores.




#### e. ¿Cuáles son los productos más monitoreados en cada entidad?

In [15]:
from pyspark.sql.functions import desc
df.groupBy('estado','producto').count().sort(desc('count')).show()

+----------------+--------------------+------+
|          estado|            producto| count|
+----------------+--------------------+------+
|DISTRITO FEDERAL|            REFRESCO|287463|
|DISTRITO FEDERAL|                 FUD|207569|
|          MÉXICO|            REFRESCO|194939|
|DISTRITO FEDERAL|LECHE ULTRAPASTEU...|175640|
|DISTRITO FEDERAL|   DETERGENTE P/ROPA|173452|
|          MÉXICO|                 FUD|149141|
|DISTRITO FEDERAL|             YOGHURT|136720|
|DISTRITO FEDERAL|             CERVEZA|136686|
|          MÉXICO|   DETERGENTE P/ROPA|132862|
|DISTRITO FEDERAL|            MAYONESA|131103|
|DISTRITO FEDERAL|      CHILES EN LATA|130598|
|DISTRITO FEDERAL|    JABON DE TOCADOR|129889|
|DISTRITO FEDERAL|             SHAMPOO|125603|
|DISTRITO FEDERAL|      LECHE EN POLVO|116827|
|          MÉXICO|LECHE ULTRAPASTEU...|116522|
|DISTRITO FEDERAL|         DESODORANTE|113779|
|DISTRITO FEDERAL|               HUEVO|111412|
|DISTRITO FEDERAL|     TOALLA FEMENINA|102356|
|DISTRITO FED

#### f. ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

In [20]:
df.groupBy('cadenaComercial','producto').count().sort(desc('cadenaComercial')).show()

+----------------+--------------------+-----+
| cadenaComercial|            producto|count|
+----------------+--------------------+-----+
| cadenaComercial|            producto|   20|
|ZARAGOZA MUEBLES| HORNO DE MICROONDAS|   62|
|ZARAGOZA MUEBLES|           CAFETERAS|   18|
|ZARAGOZA MUEBLES|     HORNO ELECTRICO|   17|
|ZARAGOZA MUEBLES|      REFRIGERADORES|   71|
|ZARAGOZA MUEBLES|COMPONENTES DE AUDIO|   47|
|ZARAGOZA MUEBLES|   TOSTADORES DE PAN|   16|
|ZARAGOZA MUEBLES|     RADIOGRABADORAS|   56|
|ZARAGOZA MUEBLES|        VIDEOCAMARAS|   41|
|ZARAGOZA MUEBLES|           LAVADORAS|  107|
|ZARAGOZA MUEBLES|EXTRACTORES DE JU...|   38|
|ZARAGOZA MUEBLES|   CAMARAS DIGITALES|   26|
|ZARAGOZA MUEBLES|             ESTUFAS|   50|
|ZARAGOZA MUEBLES|          LICUADORAS|   19|
|ZARAGOZA MUEBLES|         TELEVISORES|   55|
|ZARAGOZA MUEBLES|            PLANCHAS|   44|
| ZAPOPAN ESCOLAR|CUADERNO FORMA CO...|  165|
| ZAPOPAN ESCOLAR|         COLOR LARGO| 2299|
| ZAPOPAN ESCOLAR|              CO

### Análisis exploratorio

#### a. Genera una canasta de productos básicos que te permita comparar los precios geográfica y temporalmente. Justifica tu elección y procedimiento

De acuerdo al Gobierno de México, actualmente existen 40 productos de la canasta básica. Por lo que, con base en ello se seleccionaron algunos productos intentando que haya al menos uno de cada grupo de alimentos:
#### - Leche y derivados
- Leche
#### - Carnes, pescados y huevos
- Pollo
- Res
- Puerco
- Huevo

#### -Patatas, legumbres, frutos secos
- Frijoles
- Lentejas


#### - Verduras y hortalizas
- Zanahoria
- Aguacate
- Chayote
- Calabacitas


#### - Frutas
- Plátano
- Jitomate
 
#### - Cereales y derivados, azúcares y dulces

- Chocolate
- Pasta para sopa

#### - Grasas, aceite y mantequilla

- Aceite vegetal comestible


Y con base a lo publicado por la CONASAMI, el salario mínimo está fijado en 123.22 pesos diarios, por lo que el rango de esta selecicón estaría limitada a este ingreso mensual, se tendrían que considerar los productos de acuerdo a si alcanza o no a comprarse con el ingreso. Con la selección de estos productos básicos, se garantiza su aparición en el inventario de las tiendas por lo que es más fácil trazarlo en el tiempo.


#### c. ¿Hay algún patrón estacional entre años?



In [24]:
data_df =df.withColumn('precio', df.precio.cast("double"))

In [27]:
canasta_basica = ['LECHE','POLLO','FILETE DE RES','PUERCO','HUEVO','FRIJOLES','LENTEJAS','ZANAHORIA','AGUACATE', 'CHAYOTE','CALABACITAS','PLATANO','JITOMATE', 'CHOCOLATE','PASTA']

In [40]:
from pyspark.sql.functions import col
canasta_filtro = data_df.filter(col('producto').isin(canasta_basica))

In [41]:
canasta_filtro.show()

+---------+--------------------+----------+--------------------+------------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+---------+----------+
| producto|        presentacion|     marca|           categoria|          catalogo|precio|       fechaRegistro|   cadenaComercial|                giro|     nombreComercial|           direccion|          estado|           municipio|  latitud|  longitud|
+---------+--------------------+----------+--------------------+------------------+------+--------------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+---------+----------+
|ZANAHORIA|        1 KG. GRANDE|       S/M|  HORTALIZAS FRESCAS|FRUTAS Y LEGUMBRES|  12.9|2011-01-10 00:00:...|COMERCIAL MEXICANA|TIENDA DE AUTOSER...|COMERCIAL MEXICAN...|AV. LAGO DE GUADA...|          MÉXICO|ATIZAPAN         ...|       NA|

#### b. ¿Cuál es la ciudad más cara del país? ¿Cuál es la más barata?

De acuerdo al Índice de Precios al Consumidor (INPC) emitido por el Instituto Nacional de Estadística y Geografía (INEGI) la ciudad de __Puebla__ es la ciudad más cara, al contrario de __Querétaro__ que es la más barata.

In [47]:

canasta_filtro.groupBy('estado').sum().sort(desc('sum(precio)')).show()

+--------------------+------------------+
|              estado|       sum(precio)|
+--------------------+------------------+
|    DISTRITO FEDERAL| 4859810.320000046|
|              MÉXICO|3188114.3200000087|
|             JALISCO|1854483.1199999966|
|          NUEVO LEÓN| 794540.4099999983|
|          GUANAJUATO| 643610.0699999998|
|              PUEBLA| 634233.0200000001|
| MICHOACÁN DE OCAMPO| 523057.3800000001|
|            TLAXCALA| 518321.3300000001|
|              SONORA|482641.56000000023|
|        QUINTANA ROO|482398.91000000027|
|           QUERÉTARO| 463411.8700000003|
|             SINALOA| 451903.6200000005|
|             TABASCO| 430197.9200000003|
|COAHUILA DE ZARAGOZA|         426506.27|
|     BAJA CALIFORNIA|400075.87999999966|
|             YUCATÁN|357914.34000000014|
|          TAMAULIPAS|343265.91999999946|
| BAJA CALIFORNIA SUR|334276.94999999995|
|             HIDALGO| 315811.3800000001|
|           ZACATECAS| 310293.9800000003|
+--------------------+------------

In [48]:
canasta_filtro.groupBy('estado').sum().sort('sum(precio)').show()

+--------------------+------------------+
|              estado|       sum(precio)|
+--------------------+------------------+
| COL. EDUARDO GUERRA|           5677.35|
|                null| 9940.539999999997|
|             NAYARIT|          70070.16|
|            CAMPECHE|123522.98000000005|
|             DURANGO|125638.61000000002|
|             CHIAPAS|153819.41000000012|
|     SAN LUIS POTOSÍ|182003.39000000013|
|            GUERRERO| 185324.9300000001|
|             MORELOS|189372.21000000005|
|      AGUASCALIENTES| 223080.2100000002|
|           CHIHUAHUA|229642.06000000017|
|VERACRUZ DE IGNAC...| 250872.3100000001|
|              OAXACA|263969.14000000025|
|              COLIMA| 277549.8500000001|
|           ZACATECAS| 310293.9800000003|
|             HIDALGO| 315811.3800000001|
| BAJA CALIFORNIA SUR|334276.94999999995|
|          TAMAULIPAS|343265.91999999946|
|             YUCATÁN|357914.34000000014|
|     BAJA CALIFORNIA|400075.87999999966|
+--------------------+------------